In [1]:
import pandas as pd
import requests
from abc import ABCMeta, abstractmethod
from bs4 import BeautifulSoup, NavigableString, Tag


# ----------------------------
# Utility
# ----------------------------

# Soup化
def getHtmltoSoup(url, proxy=None):
    res = requests.get(url, proxies=proxy)
    soup = BeautifulSoup(res.content, "html.parser")
    return soup

# カンマ区切りリストをDataFrameに変換
def setDF(tree) -> pd.DataFrame:
    series = pd.Series(tree.getContents())
    rows = series.str.split('\r\n', expand=True).T
    df = rows[0].str.split(',', expand=True)
    
    df.columns = ['Category', 'Number', 'Title', 'Question', 'Answer']# ヘッダー名を設定
    df = df.iloc[:, [1,0,2,3,4]]# 列いれかえ
    return df

# 後処理
def cleansingDF(df, columns=['Question', 'Answer']):
    for _, col in enumerate(columns):
        df[col] = df[col].apply(lambda x: x.replace('\n', '<br>')) #改行のhtmlタグ置換
        df[col] = df[col].apply(lambda x: x.replace(r'\u3000', '')) #全角スペース削除
        df[col] = df[col].apply(lambda x: x.replace('\t', '')) #tab削除
        df[col] = df[col].apply(lambda x: x.strip())
    return df

# ----------------------------
# Class
# ----------------------------

# Component Class
class Content(metaclass=ABCMeta):
    @abstractmethod
    def getContents(self):
        pass

    #
    # private
    #
    # 改行コードを潰す
    def _scrubCrLf(self, str):
        return str.replace("\r\n", "").replace("\n", "")

# Composite Class
class SiteTree(Content):
    # @return SiteTree
    @staticmethod
    def create(siteParser, soup):
        category_list = siteParser.page2CategoryList(soup)
        return SiteTree(category_list)

    def __init__(self, category_list):
        self.category_list = category_list
 
    def getContents(self):
        return "\r\n".join(map(lambda x: x.getContents(), self.category_list))

# Composite Class
class Category(Content):
    # @return Category
    @staticmethod
    def create(siteParser, tag_list):
        title = siteParser.category2title(tag_list)
        question_list = siteParser.category2QuestionList(tag_list)
        return Category(title, question_list)

    def __init__(self, title, question_list):
        self.title = title
        self.question_list = question_list
 
    def getContents(self):
        return "\r\n".join(map(lambda x: self.title + "," + x.getContents(), self.question_list))

# Composite Class
class Question(Content):
    _id = 0

    @staticmethod
    def createUniqueId():
        Question._id += 1
        return Question._id

    # @return Question
    @staticmethod
    def create(siteParser, tag_list):
        id = Question.createUniqueId()
        title, body = siteParser.question2Body(tag_list)
        answer = siteParser.question2Answer(tag_list)
        return Question(id, title, body, answer)

    def __init__(self, id, title, body, answer):
        self.id = id 
        self.title = title
        self.body = self._scrubCrLf(body)
        self.answer = answer

    def getContents(self):
        return ",".join([str(self.id),
                          self.title,
                          self.body,
                          self.answer.getContents()])

# Leaf Class
class Answer(Content):
    # @return Answer
    @staticmethod
    def create(siteParser, tag_list):
        body = siteParser.answer2Body(tag_list)
        return Answer(body)

    def __init__(self, body):
        self.body = self._scrubCrLf(body)

    def getContents(self):
        return ",".join([self.body])


class SiteParser(metaclass=ABCMeta):
    # @return String
    @abstractmethod
    def getPageURL(self):
        pass

    # @return List<Category>
    # param soup: ページ全体のSoup
    @abstractmethod
    def page2CategoryList(self, soup):
        pass

    # @return String
    # param tag_list: 1Category分のList<Tag>
    @abstractmethod
    def category2title(self, tag_list):
        pass

    # @return List<Question>
    # param tag_list: 1Category分のList<Tag>
    @abstractmethod
    def category2QuestionList(self, tag_list):
        pass

    # @return String
    # param tag_list: 1Question分のList<Tag>
    @abstractmethod
    def question2Body(self, tag_list):
        pass

    # @return Answer
    # param tag_list: 1Question分のList<Tag>
    @abstractmethod
    def question2Answer(self, tag_list):
        pass

    # @return String
    # param tag_list: 1Answer分のList<Tag>
    @abstractmethod
    def answer2Body(self, tag_list):
        pass
    
    
# ----------------------------
# 群馬県(サイト固有のスクレイピング処理を書く)
# ----------------------------
class SiteGumma(SiteParser):
    # エンドポイント
    def getPageURL(self):
        return 'https://www.pref.gunma.jp/03/x0110028.html'
    #
    # page
    #
    def page2CategoryList(self, soup):
        tag_list = self._extractTargetTagList(soup)
        category_list, buf = [], []
        for t in tag_list:
            if t.name == "h3" and buf: # h3要素がcategoryのタイトル
                category_list.append(Category.create(self, buf))
                buf = []
            buf.append(t)
        if buf:
            category_list.append(Category.create(self, buf)) # 最後の要素をケア
        return category_list
    
    # ページ全体から必要な部分だけを取ってくる
    def _extractTargetTagList(self, soup):
        soup = soup.select('#scroll-main > .typeA > .honbun')
        tag_list = list(filter(lambda x: x.name != None, soup[0]))
        buf = []
        skip = True
        for t in tag_list:
            if not skip:
                buf.append(t)
            # よくある質問と回答まで読み飛ばす
            if skip and t.string and "よくある質問と回答" in t.string:
                skip = False
            if t.name == "hr" and buf: # ココまで
                break
        return buf

    #
    # Category
    #
    def category2title(self, tag_list):
        return tag_list[0].string

    def category2QuestionList(self, tag_list):
        del tag_list[0] # categoryのタイトルを飛ばす
        question_list, buf = [], []
        for t in tag_list:
            if t.name == "h4" and buf: # h4要素がquestionのタイトル
                question_list.append(Question.create(self, buf))
                buf = []
            buf.append(t)
        if buf: # 最後の要素をケア
            question_list.append(Question.create(self, buf))
        return question_list

    #
    # Question
    #
    def question2Body(self, tag_list):
        buf = []
        for t in tag_list:
            buf.append(t)
        return tag_list[0].string, "".join(map(str, buf))

    def question2Answer(self, tag_list):
        del tag_list[0] # titleを飛ばす
        return Answer.create(self, tag_list)
    
    #
    # Answer
    #
    def answer2Body(self, tag_list):
        buf = []
        for t in tag_list:
            buf.append(t)
        return "".join(map(str, buf))


### ここからmain ###
if __name__ == '__main__':
    parser = SiteGumma() # サイトによってこれを変える

    soup = getHtmltoSoup(parser.getPageURL())
    tree = SiteTree.create(parser, soup)
    # treeの中身確認↓
    # print(tree.category_list[2].question_list[3].getContents())
    
    df = cleansingDF(setDF(tree))
    df.to_csv('./faq_gunma.csv', encoding='utf-8', index=None)
    
df

,Number,Category,Title,Question,Answer
0,1,１ 教員採用試験,１－１ 教員採用試験を受験したいのですが、提出書類はどこで配付していますか。,"<h4 id=""1-1haifu"">１－１ 教員採用試験を受験したいのですが、提出書類はどこ...",<p>（問い合わせ先電話番号）小中学校：学校人事課 027-226-4593、高等学校・特別...
1,2,１ 教員採用試験,１－２ 教員採用試験の過去問題は閲覧できますか。,"<h4 id=""1-2kakomondaieturan"">１－２ 教員採用試験の過去問題は閲...",<p>（問い合わせ先電話番号）小中学校：学校人事課 027-226-4593、高等学校・特別...
2,3,１ 教員採用試験,１－３ 教員採用試験で、所有している免許状が専修、1種と2種では採用試験や働く上での違いがあ...,"<h4 id=""1-4hatarakuuenotigai"">１－３ 教員採用試験で、所有して...",<p>（問い合わせ先電話番号）小中学校：学校人事課 027-226-4593、高等学校・特別...
3,4,１ 教員採用試験,１－４ 私は日本国籍ではないのですが、教員として採用されますか。,"<h4 id=""1-5nihonkokusekidenai"">１－４ 私は日本国籍ではないの...",<p>（問い合わせ先電話番号）小中学校：学校人事課 027-226-4593、高等学校・特別...
4,5,２ 教員免許,２－１ 教員免許状を取得したいのですが、どうすればよいでしょうか。,"<h4 id=""2-1syutoku"">２－１ 教員免許状を取得したいのですが、どうすればよ...",<p>（問い合わせ先電話番号）学校人事課 027-226-4602</p><p> 群馬県内に...
5,6,２ 教員免許,２－２ 教員免許状を紛失してしまいました。再交付をしてもらえるのでしょうか。,"<h4 id=""2-3funsitu"">２－２ 教員免許状を紛失してしまいました。再交付をし...",<p>（問い合わせ先電話番号）学校人事課 027-226-4602</p><p> 群馬県教育...
6,7,２ 教員免許,２－３ 教員免許更新制とは、どんな制度か教えてください。,"<h4 id=""2-4kousinsei"">２－３ 教員免許更新制とは、どんな制度か教えてく...",<p>（問い合わせ先電話番号）学校人事課 027-226-4602</p><p> 教員免許更...
7,8,２ 教員免許,２－４ 今までに教職経験はありません。また、今後教職に就く予定もないのですが、持っている免許...,"<h4 id=""2-5tetuduki"">２－４ 今までに教職経験はありません。また、今後教...",<p>（問い合わせ先電話番号）学校人事課 027-226-4602</p><p> 教員免許更...
8,9,２ 教員免許,２－５ 教員免許更新の申請をしたいのですが、手続きや申請書類の入手方法を教えてください。,"<h4 id=""2-6kousin"">２－５ 教員免許更新の申請をしたいのですが、手続きや申...",<p>（問い合わせ先電話番号）学校人事課 027-226-4602</p><p> 群馬県内に...
9,10,３ 県立学校入試,３－１ 高校入試（県立学校の令和3年度入学者選抜）の日程を教えてください。,"<h4 id=""3-1koukounyusi"">３－１ 高校入試（県立学校の令和3年度入学者...",<p>（問い合わせ先電話番号）高校教育課 027-226-4647</p><p><a hre...
